In [3]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

class TrieNode:
    def __init__(self):
        self.children = defaultdict(TrieNode)
        self.is_end = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, conversation):
        node = self.root
        for tweet_id in conversation:
            node = node.children[tweet_id]
        node.is_end = True

    def is_subset(self, conversation):
        node = self.root
        for tweet_id in conversation:
            if tweet_id not in node.children:
                return False
            node = node.children[tweet_id]
        return True

def trace_conversation(start_tweet_id: str, tweet_dict: dict):
    convo = []
    current_tweet_id = start_tweet_id
    users_in_conversation = set()
    local_processed_tweet_ids = set()  # Local set to track the current conversation
    while current_tweet_id:
        if current_tweet_id not in tweet_dict or current_tweet_id in local_processed_tweet_ids:
            break
        tweet_info = tweet_dict[current_tweet_id]
        convo.append(current_tweet_id)
        users_in_conversation.add(tweet_info['user_id'])
        local_processed_tweet_ids.add(current_tweet_id)
        if len(users_in_conversation) > 2:
            return convo[:-1][::-1]  # As soon as the third user appears, we delete his tweet and return
        current_tweet_id = tweet_info['replied_tweet_id']
    return convo[::-1] if len(users_in_conversation) == 2 else None

def extract_and_filter_conversations(df: pd.DataFrame):
    df = df.sort_values("tweet_creation_time", ascending=False)
    df.index = df.index.astype(str)
    tweet_dict = df.to_dict('index')
    conversations = []
    trie = Trie()  # Initialize trie for subset checks

    # Start tracing conversations from tweets that are replies
    for tweet_id in tqdm(df[df['replied_tweet_id'].notnull()].index, desc="Extracting all conversations"):
        if conversation := trace_conversation(tweet_id, tweet_dict):
            if not trie.is_subset(conversation):
                trie.insert(conversation)
                conversations.append(conversation)

    return conversations

# Define the test data
test_cases = [
    {
        'name': '1. Simple Conversation Between User and Airline',
        'data': {
            'tweet_id': ['1', '2', '3', '4'],
            'user_id': ['airline', 'user_a', 'airline', 'user_a'],
            'replied_tweet_id': [None, '1', '2', '3'],
            'tweet_creation_time': ['2023-05-01T10:00:00Z', '2023-05-01T10:05:00Z', '2023-05-01T10:10:00Z', '2023-05-01T10:15:00Z']
        },
        'expected': [['1', '2', '3', '4']]
    },
    {
        'name': '2. User-Initiated Conversation',
        'data': {
            'tweet_id': ['1', '2', '3'],
            'user_id': ['user_a', 'airline', 'user_a'],
            'replied_tweet_id': [None, '1', '2'],
            'tweet_creation_time': ['2023-05-01T11:00:00Z', '2023-05-01T11:05:00Z', '2023-05-01T11:10:00Z']
        },
        'expected': [['1', '2', '3']]
    },
    {
        'name': '3. More Than Two Users Involved',
        'data': {
            'tweet_id': ['1', '2', '3', '4'],
            'user_id': ['user_a', 'airline', 'user_b', 'airline'],
            'replied_tweet_id': [None, '1', '2', '3'],
            'tweet_creation_time': ['2023-05-01T12:00:00Z', '2023-05-01T12:05:00Z', '2023-05-01T12:10:00Z', '2023-05-01T12:15:00Z']
        },
        'expected': [['1', '2'], ['2', '3', '4']]
    },
    {
        'name': '4. Conversation Branches Out',
        'data': {
            'tweet_id': ['1', '2', '3', '4', '5'],
            'user_id': ['airline', 'user_a', 'airline', 'user_a', 'user_b'],
            'replied_tweet_id': [None, '1', '2', '3', '2'],
            'tweet_creation_time': ['2023-05-01T13:00:00Z', '2023-05-01T13:05:00Z',
                                    '2023-05-01T13:10:00Z', '2023-05-01T13:15:00Z', '2023-05-01T13:20:00Z']
        },
        'expected': [['2', '5'], ['1', '2', '3', '4']]
    },
    {
        'name': '5. Non-Reply Initial Tweet by Airline',
        'data': {
            'tweet_id': ['1', '2'],
            'user_id': ['airline', 'user_a'],
            'replied_tweet_id': [None, '1'],
            'tweet_creation_time': ['2023-05-01T14:00:00Z', '2023-05-01T14:05:00Z']
        },
        'expected': [['1', '2']]
    },
    {
        'name': '6. Non-Reply Initial Tweet by User',
        'data': {
            'tweet_id': ['1', '2'],
            'user_id': ['user_a', 'airline'],
            'replied_tweet_id': [None, '1'],
            'tweet_creation_time': ['2023-05-01T15:00:00Z', '2023-05-01T15:05:00Z']
        },
        'expected': [['1', '2']]
    },
    # Might be helpful for future, but duplicate tweet_id is impossible
    # {
    #     'name': '7. Conversation with Duplicate Tweets',
    #     'data': {
    #         'tweet_id': ['1', '2', '3', '2'],
    #         'user_id': ['airline', 'user_a', 'airline', 'user_a'],
    #         'replied_tweet_id': [None, '1', '2', '3'],
    #         'tweet_creation_time': ['2023-05-01T16:00:00Z', '2023-05-01T16:05:00Z', '2023-05-01T16:10:00Z', '2023-05-01T16:15:00Z']
    #     },
    #     'expected': [['1', '2', '3', '2']]
    # },
    {
        'name': '8. Same User Replies to Themselves',
        'data': {
            'tweet_id': ['1', '2', '3'],
            'user_id': ['user_a', 'user_a', 'user_a'],
            'replied_tweet_id': [None, '1', '2'],
            'tweet_creation_time': ['2023-05-01T17:00:00Z', '2023-05-01T17:05:00Z', '2023-05-01T17:10:00Z']
        },
        'expected': []
    },
    {
        'name': '9. Mixed Order of Tweets',
        'data': {
            'tweet_id': ['3', '1', '2'],
            'user_id': ['user_a', 'airline', 'user_a'],
            'replied_tweet_id': [None, '3', '1'],
            'tweet_creation_time': ['2023-05-01T18:00:00Z', '2023-05-01T18:05:00Z', '2023-05-01T18:10:00Z']
        },
        'expected': [['3', '1', '2']]
    },
    {
        'name': '10. Multiple Independent Conversations',
        'data': {
            'tweet_id': ['1', '2', '3', '4', '5', '6'],
            'user_id': ['airline', 'user_a', 'airline', 'user_b', 'airline', 'user_a'],
            'replied_tweet_id': [None, '1', None, '3', None, '5'],
            'tweet_creation_time': ['2023-05-01T19:00:00Z', '2023-05-01T19:05:00Z', '2023-05-01T19:10:00Z', 
                                    '2023-05-01T19:15:00Z', '2023-05-01T19:20:00Z', '2023-05-01T19:25:00Z']
        },
        'expected': [['1', '2'], ['3', '4'], ['5', '6']]
    },
    {
        'name': '11. Nested Replies',
        'data': {
            'tweet_id': ['1', '2', '3', '4'],
            'user_id': ['airline', 'user_a', 'airline', 'user_a'],
            'replied_tweet_id': [None, '1', '2', '3'],
            'tweet_creation_time': ['2023-05-01T20:00:00Z', '2023-05-01T20:05:00Z', '2023-05-01T20:10:00Z', '2023-05-01T20:15:00Z']
        },
        'expected': [['1', '2', '3', '4']]
    },
]

# Run the test cases
for test_case in test_cases:
    df = pd.DataFrame(test_case['data']).set_index('tweet_id')
    df["tweet_creation_time"] = pd.to_datetime(df["tweet_creation_time"])
    print(df)
    result = extract_and_filter_conversations(df)
    assert sorted(result) == sorted(test_case['expected']), f"'{test_case['name']}' failed: expected {test_case['expected']}, got {result}"
    print(f"'{test_case['name']}' passed.")
    print()
print("All tests passed!")

          user_id replied_tweet_id       tweet_creation_time
tweet_id                                                    
1         airline             None 2023-05-01 10:00:00+00:00
2          user_a                1 2023-05-01 10:05:00+00:00
3         airline                2 2023-05-01 10:10:00+00:00
4          user_a                3 2023-05-01 10:15:00+00:00


Extracting all conversations: 100%|██████████| 3/3 [00:00<?, ?it/s]


'1. Simple Conversation Between User and Airline' passed.

          user_id replied_tweet_id       tweet_creation_time
tweet_id                                                    
1          user_a             None 2023-05-01 11:00:00+00:00
2         airline                1 2023-05-01 11:05:00+00:00
3          user_a                2 2023-05-01 11:10:00+00:00


Extracting all conversations: 100%|██████████| 2/2 [00:00<00:00, 1971.93it/s]


'2. User-Initiated Conversation' passed.

          user_id replied_tweet_id       tweet_creation_time
tweet_id                                                    
1          user_a             None 2023-05-01 12:00:00+00:00
2         airline                1 2023-05-01 12:05:00+00:00
3          user_b                2 2023-05-01 12:10:00+00:00
4         airline                3 2023-05-01 12:15:00+00:00


...

'9. Mixed Order of Tweets' passed.

          user_id replied_tweet_id       tweet_creation_time
tweet_id                                                    
1         airline             None 2023-05-01 19:00:00+00:00
2          user_a                1 2023-05-01 19:05:00+00:00
3         airline             None 2023-05-01 19:10:00+00:00
4          user_b                3 2023-05-01 19:15:00+00:00
5         airline             None 2023-05-01 19:20:00+00:00
6          user_a                5 2023-05-01 19:25:00+00:00


Extracting all conversations: 100%|██████████| 3/3 [00:00<?, ?it/s]


'10. Multiple Independent Conversations' passed.

          user_id replied_tweet_id       tweet_creation_time
tweet_id                                                    
1         airline             None 2023-05-01 20:00:00+00:00
2          user_a                1 2023-05-01 20:05:00+00:00
3         airline                2 2023-05-01 20:10:00+00:00
4          user_a                3 2023-05-01 20:15:00+00:00


Extracting all conversations: 100%|██████████| 3/3 [00:00<?, ?it/s]

'11. Nested Replies' passed.

All tests passed!
